# Voxel VAE-GAN Training

This notebook is designed to provide a wholistic vae-gan training experience. You can adjust the model and training parameters through the sacred configuration file, you can view training progress in tensorboard, and you can (wip) create reconstructions with the saved models!

References:

* https://github.com/anitan0925/vaegan/blob/master/examples/train.py
  * Runs 20 epochs on separate VAE and GAN then 200 on VAEGAN
* https://github.com/jlindsey15/VAEGAN/blob/master/main.py
  * Almost clear code for vaegan paper
* https://arxiv.org/pdf/1512.09300.pdf
  * vaegan paper
* https://github.com/timsainb/Tensorflow-MultiGPU-VAE-GAN
  * Best code yet!
  

## Setup

In [1]:
import env
from train_vaegan import train_vaegan
from data.thingi10k import Thingi10k
from data.modelnet10 import ModelNet10
from data import MODELNET10_TOILET_INDEX, MODELNET10_SOFA_INDEX, MODELNET10_SOFA_TOILET_INDEX
from models import MODEL_DIR


# plot things
%matplotlib inline
# autoreload modules
%load_ext autoreload
%autoreload 2

## Prepare Sacred Experiment

In [2]:
from sacred.observers import FileStorageObserver
from sacred import Experiment
import os

ex = Experiment(name='voxel_vaegan_notebook', interactive=True)
ex.observers.append(FileStorageObserver.create('experiments_vaegan'))

@ex.main
def run_experiment(cfg):
    train_vaegan(cfg)

import datetime
last_model_dir = None

## Prepare Model Config

The model dir is generated with a timestamp. This keeps you from overwriting past results and keeps results separate to avoid confusing tensorboard.

But be warned! These model dirs can take up space, so you might need to periodically go back and delete ones you do not care about.

Also, if you ever train a model that you would really like to keep, I recommend moving it to a new directory with a special name like "best_model_ever".

In [3]:
DATASET_CLASS = 'ModelNet10'
#INDEX = MODELNET10_SOFA_TOILET_INDEX
#INDEX = MODELNET10_SOFA_INDEX
INDEX = MODELNET10_TOILET_INDEX

def make_cfg():
    model_dir = os.path.join(
        MODEL_DIR,
        'voxel_vaegan1/modelnet10/{}'.format(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')))
    print(model_dir)
    os.makedirs(model_dir)

    cfg = {
        'cfg': {
            "dataset": {
                "class": DATASET_CLASS,
                "index": INDEX,
                #"tag": "animal",
                #"filter_id": 126660,
                #"pctile": 1.0,
                "splits": True
                #"splits": {
                #    "train": .8,
                #    "dev": .1,
                #    "test": .1
                #}
            },
            "generator": {
                "verbose": True,
                "pad": True
            }, 
            "model": {
                "ckpt_dir": model_dir,
                "voxels_dim": 32,
                "batch_size": 32,
                # Do 0.0001 for 1 epoch, then 0.001 for rest of training
                #"learning_rate": [(1, 0.0001), (None, 0.001)],
                #"learning_rate": 0.0001,
                "enc_lr": 0.0001,
                "dec_lr": 0.0001,
                "dis_lr": 0.0001,
                "epochs": 201,
                "keep_prob": 1.0,
                "kl_div_loss_weight": 100,
                "recon_loss_weight": 10000,
                "ll_weight": 0.001,
                "dec_weight": 100,
                "latent_dim": 100,
                "verbose": True,
                "debug": False,
                "input_repeats": 1,
                "display_step": 1,
                #"example_stl_id": 126660,
                "voxel_prob_threshold": 0.065,
                "dev_step": 10,
                "save_step": 10,
                'launch_tensorboard': True,
                'tb_dir': 'tb',
                #'tb_compare': [('best_sofa_and_toilet', '/home/jcworkma/jack/3d-form/models/voxel_vaegan1/modelnet10/2019-03-15_17-08-43/tb')],
                #'tb_compare': [('best_vaegan', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-17_08-40-29/tb')],
                'tb_compare': [('vaegan_100epochs_toilets', '/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-18_13-12-53/tb')],
                'no_gan': False,
                'monitor_memory': True
            }
        }
    }
    
    return cfg

## Tensorboard Prep

We launch tensorboard with a call to the python subprocess module. Sometimes, that process does not die with the rest of the experiment and lingers on as a system process. This becomes a problem when we try to initialize tensorboard for the next experiment because they cannot share the same port!

The function below is designed to solve this problem. It uses the linux pgrep utility to search for existing tensorboard processes and kill them. Note that this probably won't work on Windows.

In [4]:
from utils import kill_tensorboard

kill_tensorboard()

['pgrep', 'tensorboard'] yielded -> b'13746\n'
killed b'13746'!


## Training

We start with a check that we are not attempting to overwrite the last MODEL_DIR. If you are blocked by the assert, re-execute the cfg code above to generate a new MODEL_DIR. This will allow you to move ahead with training.

The sacred experiment will save away a copy of your experiment settings in an experiments directory. This can be accessed later in case we need to retrieve a prime config.

If tensorboard is enabled, tune in at localhost:6006 or your_ip:6006
   

In [ ]:
cfg = make_cfg()
model_dir = cfg.get('cfg').get('model').get('ckpt_dir')
kill_tensorboard()

/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-19_14-25-04
['pgrep', 'tensorboard'] yielded -> b''


In [ ]:
if last_model_dir == model_dir:
    print('dont overwrite!')
    assert False
else:
    last_model_dir = model_dir

ex.run(config_updates=cfg)

WARNING - root - Added new config entry: "cfg.dataset.class"
WARNING - root - Added new config entry: "cfg.dataset.index"
WARNING - root - Added new config entry: "cfg.dataset.splits"
WARNING - root - Added new config entry: "cfg.generator.pad"
WARNING - root - Added new config entry: "cfg.generator.verbose"
WARNING - root - Added new config entry: "cfg.model.batch_size"
WARNING - root - Added new config entry: "cfg.model.ckpt_dir"
WARNING - root - Added new config entry: "cfg.model.debug"
WARNING - root - Added new config entry: "cfg.model.dec_lr"
WARNING - root - Added new config entry: "cfg.model.dec_weight"
WARNING - root - Added new config entry: "cfg.model.dev_step"
WARNING - root - Added new config entry: "cfg.model.dis_lr"
WARNING - root - Added new config entry: "cfg.model.display_step"
WARNING - root - Added new config entry: "cfg.model.enc_lr"
WARNING - root - Added new config entry: "cfg.model.epochs"
WARNING - root - Added new config entry: "cfg.model.input_repeats"
WARNIN

Logging to /home/jcworkma/jack/3d-form/src/logs/2019-03-19_14-25__root.log
Starting train_vaegan main
Numpy random seed: 980464534
Saved cfg: /home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-19_14-25-04/cfg.json
Dataset: <class 'data.modelnet10.ModelNet10'>
Using dataset index /home/jcworkma/jack/3d-form/src/../data/processed/modelnet10_toilet_index.csv and pctile None
Shuffling dataset
dataset n_input=7104
Splitting Datasets
Num input = 7104
Num batches per epoch = 222.00
Initializing VoxelVaegan
['tensorboard', '--logdir', 'current:/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-19_14-25-04/tb,vaegan_100epochs_toilets:/home/jcworkma/jack/3d-form/src/../models/voxel_vaegan1/modelnet10/2019-03-18_13-12-53/tb']
Epoch: 0, Elapsed Time: 0.03
Epoch: 0 / 201, Batch: 0 (0 / 32), Elapsed time: 0.03 mins
Enc Loss = 37.29, KL Divergence = 0.24, Reconstruction Loss = 0.12, ll_loss = 12980.60, dis_Loss = 6.58, dec_Loss = 82.30, Elapsed time: 0.

Memory Use (GB): 1.7296218872070312
Epoch: 0 / 201, Batch: 31 (0 / 1024), Elapsed time: 2.06 mins
Enc Loss = 32.41, KL Divergence = 0.20, Reconstruction Loss = 0.11, ll_loss = 11941.00, dis_Loss = 5.74, dec_Loss = 81.22, Elapsed time: 2.12 mins
Memory Use (GB): 1.8037071228027344
Epoch: 0 / 201, Batch: 32 (0 / 1056), Elapsed time: 2.12 mins
Enc Loss = 35.24, KL Divergence = 0.22, Reconstruction Loss = 0.11, ll_loss = 13180.44, dis_Loss = 7.21, dec_Loss = 82.46, Elapsed time: 2.19 mins
Memory Use (GB): 1.6386680603027344
Epoch: 0 / 201, Batch: 33 (0 / 1088), Elapsed time: 2.19 mins
Enc Loss = 35.51, KL Divergence = 0.24, Reconstruction Loss = 0.11, ll_loss = 11585.70, dis_Loss = 6.07, dec_Loss = 80.86, Elapsed time: 2.25 mins
Memory Use (GB): 1.7238845825195312
Epoch: 0 / 201, Batch: 34 (0 / 1120), Elapsed time: 2.25 mins
Enc Loss = 32.13, KL Divergence = 0.21, Reconstruction Loss = 0.11, ll_loss = 10832.40, dis_Loss = 6.68, dec_Loss = 80.11, Elapsed time: 2.32 mins
Memory Use (GB): 1.6

Memory Use (GB): 1.73834228515625
Epoch: 0 / 201, Batch: 65 (0 / 2112), Elapsed time: 4.29 mins
Enc Loss = 33.23, KL Divergence = 0.21, Reconstruction Loss = 0.10, ll_loss = 12611.96, dis_Loss = 6.79, dec_Loss = 81.89, Elapsed time: 4.36 mins
Memory Use (GB): 1.6595878601074219
Epoch: 0 / 201, Batch: 66 (0 / 2144), Elapsed time: 4.36 mins
Enc Loss = 33.19, KL Divergence = 0.21, Reconstruction Loss = 0.11, ll_loss = 12487.79, dis_Loss = 6.34, dec_Loss = 81.77, Elapsed time: 4.42 mins
Memory Use (GB): 1.6217727661132812
Epoch: 0 / 201, Batch: 67 (0 / 2176), Elapsed time: 4.43 mins
Enc Loss = 34.38, KL Divergence = 0.22, Reconstruction Loss = 0.11, ll_loss = 11941.64, dis_Loss = 6.97, dec_Loss = 81.22, Elapsed time: 4.49 mins
Memory Use (GB): 1.6208381652832031
Epoch: 0 / 201, Batch: 68 (0 / 2208), Elapsed time: 4.49 mins
Enc Loss = 32.79, KL Divergence = 0.22, Reconstruction Loss = 0.11, ll_loss = 10722.78, dis_Loss = 6.80, dec_Loss = 80.00, Elapsed time: 4.55 mins
Memory Use (GB): 1.641

Memory Use (GB): 1.7154541015625
Epoch: 0 / 201, Batch: 99 (0 / 3200), Elapsed time: 6.52 mins
Enc Loss = 30.24, KL Divergence = 0.19, Reconstruction Loss = 0.09, ll_loss = 11457.22, dis_Loss = 6.33, dec_Loss = 80.74, Elapsed time: 6.59 mins
Memory Use (GB): 1.7999076843261719
Epoch: 0 / 201, Batch: 100 (0 / 3232), Elapsed time: 6.59 mins
Enc Loss = 35.35, KL Divergence = 0.23, Reconstruction Loss = 0.12, ll_loss = 11915.82, dis_Loss = 6.62, dec_Loss = 81.19, Elapsed time: 6.65 mins
Memory Use (GB): 1.8123245239257812
Epoch: 0 / 201, Batch: 101 (0 / 3264), Elapsed time: 6.65 mins
Enc Loss = 33.71, KL Divergence = 0.20, Reconstruction Loss = 0.10, ll_loss = 13782.58, dis_Loss = 5.98, dec_Loss = 83.06, Elapsed time: 6.72 mins
Memory Use (GB): 1.6593055725097656
Epoch: 0 / 201, Batch: 102 (0 / 3296), Elapsed time: 6.72 mins
Enc Loss = 39.01, KL Divergence = 0.27, Reconstruction Loss = 0.14, ll_loss = 11747.84, dis_Loss = 5.86, dec_Loss = 81.03, Elapsed time: 6.78 mins
Memory Use (GB): 1.6

...

Memory Use (GB): 1.827880859375
Epoch: 12 / 201, Batch: 4 (160 / 5504), Elapsed time: 135.89 mins
Enc Loss = 35.69, KL Divergence = 0.21, Reconstruction Loss = 0.11, ll_loss = 14483.46, dis_Loss = 5.44, dec_Loss = 83.76, Elapsed time: 135.96 mins
Memory Use (GB): 1.7561988830566406
Epoch: 12 / 201, Batch: 5 (192 / 5504), Elapsed time: 135.96 mins
Enc Loss = 36.06, KL Divergence = 0.21, Reconstruction Loss = 0.12, ll_loss = 14621.27, dis_Loss = 5.73, dec_Loss = 83.90, Elapsed time: 136.02 mins
Memory Use (GB): 1.58685302734375
Epoch: 12 / 201, Batch: 6 (224 / 5504), Elapsed time: 136.02 mins
Enc Loss = 36.05, KL Divergence = 0.22, Reconstruction Loss = 0.12, ll_loss = 13839.97, dis_Loss = 5.30, dec_Loss = 83.12, Elapsed time: 136.09 mins
Memory Use (GB): 1.6259956359863281
Epoch: 12 / 201, Batch: 7 (256 / 5504), Elapsed time: 136.09 mins
Enc Loss = 35.33, KL Divergence = 0.20, Reconstruction Loss = 0.11, ll_loss = 14914.76, dis_Loss = 6.01, dec_Loss = 84.19, Elapsed time: 136.15 mins
Me

Memory Use (GB): 1.7155609130859375
Epoch: 12 / 201, Batch: 37 (1216 / 5504), Elapsed time: 138.06 mins
Enc Loss = 34.43, KL Divergence = 0.22, Reconstruction Loss = 0.12, ll_loss = 12929.40, dis_Loss = 5.48, dec_Loss = 82.21, Elapsed time: 138.13 mins
Memory Use (GB): 1.7912521362304688
Epoch: 12 / 201, Batch: 38 (1248 / 5504), Elapsed time: 138.13 mins
Enc Loss = 33.97, KL Divergence = 0.21, Reconstruction Loss = 0.12, ll_loss = 13020.28, dis_Loss = 5.70, dec_Loss = 82.30, Elapsed time: 138.19 mins
Memory Use (GB): 1.7601242065429688
Epoch: 12 / 201, Batch: 39 (1280 / 5504), Elapsed time: 138.19 mins
Enc Loss = 34.78, KL Divergence = 0.20, Reconstruction Loss = 0.11, ll_loss = 14507.12, dis_Loss = 6.62, dec_Loss = 83.78, Elapsed time: 138.26 mins
Memory Use (GB): 1.6269721984863281
Epoch: 12 / 201, Batch: 40 (1312 / 5504), Elapsed time: 138.26 mins
Enc Loss = 33.51, KL Divergence = 0.19, Reconstruction Loss = 0.11, ll_loss = 14649.81, dis_Loss = 5.17, dec_Loss = 83.93, Elapsed time: 

Enc Loss = 38.34, KL Divergence = 0.24, Reconstruction Loss = 0.13, ll_loss = 14159.00, dis_Loss = 5.71, dec_Loss = 83.44, Elapsed time: 140.21 mins
Memory Use (GB): 1.5890388488769531
Epoch: 12 / 201, Batch: 70 (2272 / 5504), Elapsed time: 140.21 mins
Enc Loss = 34.91, KL Divergence = 0.21, Reconstruction Loss = 0.12, ll_loss = 13727.04, dis_Loss = 4.99, dec_Loss = 83.00, Elapsed time: 140.28 mins
Memory Use (GB): 1.6118125915527344
Epoch: 12 / 201, Batch: 71 (2304 / 5504), Elapsed time: 140.28 mins
Enc Loss = 34.75, KL Divergence = 0.22, Reconstruction Loss = 0.11, ll_loss = 13118.88, dis_Loss = 5.11, dec_Loss = 82.40, Elapsed time: 140.34 mins
Memory Use (GB): 1.6309661865234375
Epoch: 12 / 201, Batch: 72 (2336 / 5504), Elapsed time: 140.34 mins
Enc Loss = 33.79, KL Divergence = 0.20, Reconstruction Loss = 0.11, ll_loss = 13349.00, dis_Loss = 5.78, dec_Loss = 82.63, Elapsed time: 140.41 mins
Memory Use (GB): 1.6860923767089844
Epoch: 12 / 201, Batch: 73 (2368 / 5504), Elapsed time: 

Memory Use (GB): 1.5776634216308594
Epoch: 12 / 201, Batch: 102 (3296 / 5504), Elapsed time: 142.31 mins
Enc Loss = 38.32, KL Divergence = 0.25, Reconstruction Loss = 0.14, ll_loss = 13598.01, dis_Loss = 5.18, dec_Loss = 82.87, Elapsed time: 142.37 mins
Memory Use (GB): 1.7391815185546875
Epoch: 12 / 201, Batch: 103 (3328 / 5504), Elapsed time: 142.37 mins
Enc Loss = 34.01, KL Divergence = 0.19, Reconstruction Loss = 0.10, ll_loss = 14963.04, dis_Loss = 5.74, dec_Loss = 84.24, Elapsed time: 142.44 mins
Memory Use (GB): 1.6069564819335938
Epoch: 12 / 201, Batch: 104 (3360 / 5504), Elapsed time: 142.44 mins
Enc Loss = 36.31, KL Divergence = 0.22, Reconstruction Loss = 0.11, ll_loss = 14774.70, dis_Loss = 4.33, dec_Loss = 84.05, Elapsed time: 142.50 mins
Memory Use (GB): 1.6368598937988281
Epoch: 12 / 201, Batch: 105 (3392 / 5504), Elapsed time: 142.50 mins
Enc Loss = 34.25, KL Divergence = 0.21, Reconstruction Loss = 0.11, ll_loss = 13663.30, dis_Loss = 5.51, dec_Loss = 82.94, Elapsed ti

Enc Loss = 32.51, KL Divergence = 0.19, Reconstruction Loss = 0.11, ll_loss = 13322.16, dis_Loss = 5.49, dec_Loss = 82.60, Elapsed time: 144.47 mins
Memory Use (GB): 1.7119941711425781
Epoch: 12 / 201, Batch: 135 (4352 / 5504), Elapsed time: 144.47 mins
Enc Loss = 34.64, KL Divergence = 0.21, Reconstruction Loss = 0.11, ll_loss = 13467.49, dis_Loss = 5.25, dec_Loss = 82.74, Elapsed time: 144.54 mins
Memory Use (GB): 1.6591453552246094
Epoch: 12 / 201, Batch: 136 (4384 / 5504), Elapsed time: 144.54 mins
Enc Loss = 32.16, KL Divergence = 0.19, Reconstruction Loss = 0.10, ll_loss = 12823.88, dis_Loss = 5.83, dec_Loss = 82.10, Elapsed time: 144.60 mins
Memory Use (GB): 1.6103706359863281
Epoch: 12 / 201, Batch: 137 (4416 / 5504), Elapsed time: 144.60 mins
Enc Loss = 33.54, KL Divergence = 0.20, Reconstruction Loss = 0.11, ll_loss = 13886.68, dis_Loss = 5.73, dec_Loss = 83.16, Elapsed time: 144.67 mins
Memory Use (GB): 1.6063880920410156
Epoch: 12 / 201, Batch: 138 (4448 / 5504), Elapsed ti